In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as salc
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json


with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo
WHERE repo_name = 'concourse' OR repo_name = 'postfacto' or repo_name = 'clarity' or repo_name = 'gpdb' or
      repo_name = 'kpack' or repo_name = 'rabbitmq-server' or repo_name = 'sonobuoy';
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

   repo_id        repo_name                              repo_path
0    26235        concourse               github.com/pcfdev-forks/
1    28051        concourse                  github.com/concourse/
2    28030         sonobuoy               github.com/vmware-tanzu/
3    27913          clarity                     github.com/vmware/
4    26983        postfacto                    github.com/pivotal/
5    27169             gpdb                github.com/pivotal-gss/
6    25857             gpdb               github.com/greenplum-db/
7    25432  rabbitmq-server                   github.com/rabbitmq/
8    26600             gpdb  github.com/Pivotal-Field-Engineering/
9    27043            kpack                    github.com/pivotal/


In [19]:
## List of repository IDs for the report
#repo_dict = {25760, 25663} #spring-boot & spring-framework
#repo_dict = {28051} # concourse
repo_dict = {26983} #postfacto
#repo_dict = {25432} #rabbitmq-server
#repo_dict = {25663} #spring-framework
#repo_dict = {28030} #sonobuoy

In [20]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
print(today)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
print(end_date)

print

start = last_month - datetime.timedelta(days=365)
year_ago = "'" + str(start) + "'"
print(year_ago)

print(last_month)
print(start)

'2020-03-12'
'2020-02-29'
'2019-03-01'
2020-02-29
2019-03-01


In [21]:
# Identifying the Longest Running Pull Requests

## Getting the Data

In [22]:
pr_all = pd.DataFrame()

for repo_id in repo_dict: 

    pr_query = salc.sql.text(f"""
                    SELECT
                        repo.repo_id AS repo_id,
                        pull_requests.pr_src_id AS pr_src_id,
                        repo.repo_name AS repo_name,
                        pr_src_author_association,
--                         repo_groups.rg_name AS repo_group,
                        pull_requests.pr_src_state,
                        pull_requests.pr_merged_at,
                        pull_requests.pr_created_at AS pr_created_at,
                        pull_requests.pr_closed_at AS pr_closed_at,
                        date_part( 'year', pr_created_at :: DATE ) AS CREATED_YEAR,
                        date_part( 'month', pr_created_at :: DATE ) AS CREATED_MONTH,
						   date_part( 'year', pr_closed_at :: DATE ) AS CLOSED_YEAR,
                        date_part( 'month', pr_closed_at :: DATE ) AS CLOSED_MONTH,
                        ( EXTRACT ( EPOCH FROM pull_requests.pr_closed_at ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_close,
                        ( EXTRACT ( EPOCH FROM pull_requests.pr_closed_at ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_close, 
                        ( EXTRACT ( EPOCH FROM first_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_first_response,
                        ( EXTRACT ( EPOCH FROM first_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_first_response, 
                        ( EXTRACT ( EPOCH FROM last_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_last_response,
                        ( EXTRACT ( EPOCH FROM last_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_last_response, 
--                         pull_requests.pr_html_url AS pr_url, pull_requests.pr_src_title AS pr_title,
                        first_response_time,
                        last_response_time,
                        average_time_between_responses,
                        assigned_count,
                        review_requested_count,
                        labeled_count,
                        subscribed_count,
                        mentioned_count,
                        referenced_count,
                        closed_count,
                        head_ref_force_pushed_count,
                        merged_count,
                        milestoned_count,
                        unlabeled_count,
                        head_ref_deleted_count,
                        comment_count--,
                       -- lines_of_code_added, 
                       -- commit_count, 
                       -- file_count
                    FROM
                        repo,
                        repo_groups,
                        pull_requests,
                        ( 
                            SELECT pull_requests.pull_request_id,
                            count(*) FILTER (WHERE action = 'assigned') AS assigned_count,
                            count(*) FILTER (WHERE action = 'review_requested') AS review_requested_count,
                            count(*) FILTER (WHERE action = 'labeled') AS labeled_count,
                            count(*) FILTER (WHERE action = 'unlabeled') AS unlabeled_count,
                            count(*) FILTER (WHERE action = 'subscribed') AS subscribed_count,
                            count(*) FILTER (WHERE action = 'mentioned') AS mentioned_count,
                            count(*) FILTER (WHERE action = 'referenced') AS referenced_count,
                            count(*) FILTER (WHERE action = 'closed') AS closed_count,
                            count(*) FILTER (WHERE action = 'head_ref_force_pushed') AS head_ref_force_pushed_count,
                            count(*) FILTER (WHERE action = 'head_ref_deleted') AS head_ref_deleted_count,
                            count(*) FILTER (WHERE action = 'milestoned') AS milestoned_count,
                            count(*) FILTER (WHERE action = 'merged') AS merged_count,
                            MIN(message.msg_timestamp) AS first_response_time,
                            COUNT(DISTINCT message.msg_timestamp) AS comment_count,
                            MAX(message.msg_timestamp) AS last_response_time,
                            (MAX(message.msg_timestamp) - MIN(message.msg_timestamp)) / COUNT(DISTINCT message.msg_timestamp) AS average_time_between_responses

                            FROM pull_request_events, pull_requests, repo, pull_request_message_ref, message
                            WHERE repo.repo_id = {repo_id}
                            AND repo.repo_id = pull_requests.repo_id
                            AND pull_requests.pull_request_id = pull_request_events.pull_request_id
                            AND pull_requests.pull_request_id = pull_request_message_ref.pull_request_id
                            AND pull_request_message_ref.msg_id = message.msg_id

                            GROUP BY pull_requests.pull_request_id
                        ) response_times--,
                        /*(
                            SELECT sum(cmt_added) AS lines_of_code_added, pull_request_id, count(DISTINCT cmt_commit_hash) AS commit_count, count(DISTINCT cmt_filename) AS file_count
                            FROM (
                                SELECT pr_cmt_id, cmt_added, pull_request_commits.pull_request_id, cmt_commit_hash, cmt_filename
                                FROM pull_request_commits, commits, pull_requests, pull_request_meta
                                WHERE cmt_commit_hash = pr_cmt_sha
                                AND pull_requests.pull_request_id = pull_request_commits.pull_request_id
                                AND pull_requests.pull_request_id = pull_request_meta.pull_request_id
                                AND pull_request_meta.pr_head_or_base = 'base' AND pr_src_meta_label LIKE '%master'
                                AND pull_requests.repo_id = {repo_id}
                                AND commits.repo_id = pull_requests.repo_id
                                AND commits.cmt_commit_hash <> pull_requests.pr_merge_commit_sha
                            ) AS pr_cmt_info
                            GROUP BY pull_request_id
                        ) counts*/
                    WHERE repo.repo_group_id = repo_groups.repo_group_id 
                        AND repo.repo_id = pull_requests.repo_id 
                        AND repo.repo_id = {repo_id} 
                        AND response_times.pull_request_id = pull_requests.pull_request_id
                       -- AND counts.pull_request_id = pull_requests.pull_request_id
                    ORDER BY
                       merged_count DESC

        """)
    pr_a = pd.read_sql(pr_query, con=engine)
    if not pr_all.empty: 
        pr_all = pd.concat([pr_all, pr_a]) 
    else: 
        # first repo
        pr_all = pr_a
display(pr_all.head(20))

,repo_id,pr_src_id,repo_name,pr_src_author_association,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,created_month,...,subscribed_count,mentioned_count,referenced_count,closed_count,head_ref_force_pushed_count,merged_count,milestoned_count,unlabeled_count,head_ref_deleted_count,comment_count
0,26983,224125345,postfacto,CONTRIBUTOR,closed,2018-11-05 15:27:32,2018-10-18 23:04:03,2018-11-05 15:27:32,2018.0,10.0,...,54,54,9,9,0,9,0,0,0,9
1,26983,246697184,postfacto,CONTRIBUTOR,closed,2019-02-15 14:27:53,2019-01-22 17:52:18,2019-02-15 14:27:53,2019.0,1.0,...,14,14,7,7,0,7,7,0,7,7
2,26983,305097679,postfacto,CONTRIBUTOR,closed,2019-10-23 12:03:32,2019-08-07 11:08:14,2019-10-23 12:03:32,2019.0,8.0,...,36,36,6,6,24,6,0,0,6,6
3,26983,255463852,postfacto,CONTRIBUTOR,closed,2019-02-25 17:51:19,2019-02-22 16:56:30,2019-02-25 17:51:19,2019.0,2.0,...,30,30,6,6,18,6,6,0,0,6
4,26983,302189832,postfacto,CONTRIBUTOR,closed,2019-09-04 07:57:16,2019-07-29 18:46:21,2019-09-04 07:57:16,2019.0,7.0,...,15,15,5,5,0,5,0,0,0,5
5,26983,275310910,postfacto,CONTRIBUTOR,closed,2019-07-11 15:39:21,2019-05-02 10:37:35,2019-07-11 15:39:21,2019.0,5.0,...,12,12,12,4,0,4,0,0,0,4
6,26983,202261195,postfacto,CONTRIBUTOR,closed,2018-07-20 07:47:21,2018-07-18 13:53:50,2018-07-20 07:47:21,2018.0,7.0,...,12,12,4,4,0,4,4,0,0,4
7,26983,299573453,postfacto,CONTRIBUTOR,closed,2019-07-22 14:33:15,2019-07-20 15:56:07,2019-07-22 14:33:15,2019.0,7.0,...,12,12,4,4,0,4,0,0,0,4
8,26983,179420773,postfacto,CONTRIBUTOR,closed,2018-04-06 09:16:38,2018-04-04 15:04:29,2018-04-06 09:16:38,2018.0,4.0,...,0,0,4,4,0,4,0,0,0,4
9,26983,307211981,postfacto,CONTRIBUTOR,closed,2019-08-15 10:35:37,2019-08-14 07:57:40,2019-08-15 10:35:37,2019.0,8.0,...,9,9,3,3,0,3,0,0,3,3


In [23]:
# Begin data pre-processing and adding columns

In [24]:
pr_all[['assigned_count',
          'review_requested_count',
          'labeled_count',
          'subscribed_count',
          'mentioned_count',
          'referenced_count',
          'closed_count',
          'head_ref_force_pushed_count',
          'merged_count',
          'milestoned_count',
          'unlabeled_count',
          'head_ref_deleted_count',
          'comment_count',
           'created_year',
           'closed_year'
       ]] = pr_all[['assigned_count',
                                      'review_requested_count',
                                      'labeled_count',
                                      'subscribed_count',
                                      'mentioned_count',
                                      'referenced_count',
                                      'closed_count',
                                        'head_ref_force_pushed_count',
                                    'merged_count',
                                      'milestoned_count',          
                                      'unlabeled_count',
                                      'head_ref_deleted_count',
                                      'comment_count',
                                        'created_year',
                                        'closed_year'
                   ]].fillna(-1).astype(int)


In [25]:
# Get days for average_time_between_responses time delta

pr_all['average_days_between_responses'] = pr_all['average_time_between_responses'].map(lambda x: x.days).astype(float)
pr_all['average_hours_between_responses'] = pr_all['average_time_between_responses'].map(lambda x: x.days * 24).astype(float)


In [26]:
# Disabled end_date filtering for now to include PRs that are still open

#start_date = pd.to_datetime('2017-07-01 01:00:00')
start_date = pd.to_datetime(start)
# end_date = pd.to_datetime('2020-02-01 09:00:00')
#end_date = pd.to_datetime('2019-12-31 23:59:59')
end_date = pd.to_datetime(last_month)
pr_all = pr_all[(pr_all['pr_created_at'] > start_date) & (pr_all['pr_closed_at'] < end_date)]

pr_all['created_year'] = pr_all['created_year'].map(int)
pr_all['created_month'] = pr_all['created_month'].map(int)
pr_all['created_month'] = pr_all['created_month'].map(lambda x: '{0:0>2}'.format(x))
pr_all['created_yearmonth'] = pd.to_datetime(pr_all['created_year'].map(str) + '-' + pr_all['created_month'].map(str) + '-01')
pr_all.head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,repo_id,pr_src_id,repo_name,pr_src_author_association,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,created_month,...,closed_count,head_ref_force_pushed_count,merged_count,milestoned_count,unlabeled_count,head_ref_deleted_count,comment_count,average_days_between_responses,average_hours_between_responses,created_yearmonth
2,26983,305097679,postfacto,CONTRIBUTOR,closed,2019-10-23 12:03:32,2019-08-07 11:08:14,2019-10-23 12:03:32,2019,08,...,6,24,6,0,0,6,6,7.0,168.0,2019-08-01
4,26983,302189832,postfacto,CONTRIBUTOR,closed,2019-09-04 07:57:16,2019-07-29 18:46:21,2019-09-04 07:57:16,2019,07,...,5,0,5,0,0,0,5,7.0,168.0,2019-07-01
5,26983,275310910,postfacto,CONTRIBUTOR,closed,2019-07-11 15:39:21,2019-05-02 10:37:35,2019-07-11 15:39:21,2019,05,...,4,0,4,0,0,0,4,16.0,384.0,2019-05-01
7,26983,299573453,postfacto,CONTRIBUTOR,closed,2019-07-22 14:33:15,2019-07-20 15:56:07,2019-07-22 14:33:15,2019,07,...,4,0,4,0,0,0,4,0.0,0.0,2019-07-01
9,26983,307211981,postfacto,CONTRIBUTOR,closed,2019-08-15 10:35:37,2019-08-14 07:57:40,2019-08-15 10:35:37,2019,08,...,3,0,3,0,0,3,3,0.0,0.0,2019-08-01
10,26983,315826040,postfacto,CONTRIBUTOR,closed,2019-10-22 09:05:45,2019-09-10 05:50:18,2019-10-22 09:05:45,2019,09,...,3,0,3,0,0,3,3,0.0,0.0,2019-09-01
11,26983,322482010,postfacto,CONTRIBUTOR,closed,2019-10-22 09:19:32,2019-09-29 07:57:40,2019-10-22 09:19:32,2019,09,...,3,0,3,0,0,3,3,7.0,168.0,2019-09-01
15,26983,288248302,postfacto,MEMBER,closed,2019-07-11 15:34:19,2019-06-14 09:19:09,2019-07-11 15:34:19,2019,06,...,3,0,3,0,0,0,3,0.0,0.0,2019-06-01
16,26983,297065260,postfacto,CONTRIBUTOR,closed,2019-07-16 15:27:42,2019-07-12 13:11:38,2019-07-16 15:27:42,2019,07,...,3,0,3,0,0,0,3,0.0,0.0,2019-07-01
17,26983,297950798,postfacto,CONTRIBUTOR,closed,2019-07-16 15:26:21,2019-07-16 09:25:00,2019-07-16 15:26:21,2019,07,...,3,0,3,0,0,3,3,0.0,0.0,2019-07-01
